In [1]:
import geopandas as gpd 
import pandas as pd
from shapely.geometry import *

In [2]:
taxi = pd.read_csv('weekend.csv')
# taxi = taxi.drop('Unnamed: 0',axis=1)
taxi.head()
taxi.rename(columns={'Longitude':'Latitude','Latitude':'Longitude'},inplace=True)

In [3]:
taxi_sub = taxi

In [4]:
taxi_sub.head()

,Latitude,Longitude,date,time
0,1.300540,103.80059,2017-03-17,18:58:56
1,1.396308,103.90990,2017-03-17,18:52:25
2,1.303450,103.76456,2017-03-17,18:57:08
3,1.358805,103.95470,2017-03-17,18:58:56
4,1.358830,103.98949,2017-03-17,18:54:20


In [5]:
airport = gpd.GeoDataFrame.from_file('airport.shp')

In [25]:
def create_geometry(df):
    geometry_arr = df.apply(lambda x: Point(x['X'], x['Y']), axis=1)
    df = df.assign(geometry=geometry_arr)
    gdf = gpd.GeoDataFrame(df)
    return gdf

In [26]:
def filter_points(pts_gpd, poly):
    index = pts_gpd.apply(lambda x: not poly.contains(x['geometry']), axis=1)
    return pts_gpd.loc[index]
    

In [8]:
test = create_geometry(taxi)

In [9]:
test2 = filter_points(test, airport['geometry'].iloc[0])

In [10]:
test2.to_file('filter_test.shp', driver='ESRI Shapefile')

In [ ]:
%matplotlib

In [12]:
gdf = pd.read_csv("../LTA_Singapore_2/data/taxi_report_15.csv")

In [27]:
new_df = create_geometry(gdf)

KeyboardInterrupt: 